![sql](images/sql-logo.jpg)

# Databases, SQL, and Connecting them to `pandas`

Long before we can do any kind of machine learning, we need access to data - most often stored in databases.

![hierarchy](images/ai-hierachy.png)

[Image Source](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

> ^ If you haven't yet read about the AI Hierarchy of Needs, I can't recommend Monica Rogati’s fantastic blog post enough! Click the image source link to give it a read.

The task of accessing data - **Extracting, Transforming and Loading** data, often shortened to **ETL** - often falls to Data Engineers, and so today we'll be wearing the hat of a Data Engineer in order to see what it means to access data where it actually is stored.

## What does it mean to **Engineer Data**?

Let's start with a basic scenario: You have *no* current database. How would you set it up?

Data-needs considerations:

- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:

- Speed of retrieval
- How much data you are accessing
- How much you are storing

### What is a Relational Database? 

> "Most databases today are relational databases, named such because they deal with tables of data related by a common field."
> - [MariaDB's Introduction to Relational Databases tutorial](https://mariadb.com/kb/en/introduction-to-relational-databases/)

From that description, we know a few things - databases are made up of tables, which contain data that is related between tables.

Let's look at an example and unpack:

![example database schema](images/example-database-schema.png)

[Image Source](http://www.cs.montana.edu/~halla/csci440/n7/n7.html)

Lots of symbols we can examine here! These **Entity-Relationship Diagrams** (ERDs) will tell you a lot about how tables relate to each other.

First lets look at the symbols along the sides of each line in there - each line is the name of a column, followed by the allowed datatype in all caps. The symbols?

- Small Key indicates a primary key
- Blue Diamond indicates a field/attribute
- Red Diamond indicates a foreign key

As for the symbols along the lines - relationship between keys! This type of notation is called Crow's Feet notation.

<img alt="database schema connectors explained" src="images/crowsfeetsymbols.png" width=400>

[Image Source](https://www.lucidchart.com/pages/ER-diagram-symbols-and-meaning)

Of course - this is but one way of visualizing these relationships (there are others)

## Enter SQL

SQL - **Structured Query Language** - is a standard language for connecting to, changing and retrieving data stored in relational databases.

SQL is a language - [but there are dialects.](https://training-nyc.com/learn/sql/the-many-flavors-of-sql-a-guide-to-relational-databases) Some are proprietary, some are open source, and all have their quirks. At their core, however, they're mostly the same and use many of the same keywords. If you're proficient in one flavor of SQL, you can easily transition to one of the others.

### SQLite

<img src="images/SQLite-Python.jpg" width=400>

One of many flavors of SQL! And the one we'll be using!

- C library (fast)
- lightweight disk-based database
- that doesn’t require a separate server process
- nonstandard variant of the SQL query language
- applications can use SQLite for internal data storage
- use case: prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle

#### Using it in Python:

[sqlite3 documentation](https://docs.python.org/2/library/sqlite3.html)

### Other flavors/database types?

To connect to other database types (thus other flavors of SQL) there are many different Python libraries:

- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

## Let's Explore a Database!

In [1]:
# of course, need an import
import pandas as pd
import sqlite3

#### Load a database object with `connect` and `cursor`

In [2]:
!ls data

Austin_Animal_Center_Intakes_10-08-20.csv
Austin_Animal_Center_Outcomes_10-14-20.csv
flight.db
flights.db
titanic.csv


In [3]:
con = sqlite3.connect('data/flights.db')
cur = con.cursor()

In [4]:
cur.execute(""" SELECT *
                FROM airlines
                LIMIT 10;
                """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df

10


,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N
5,5,6,223 Flight Unit State Airline,\N,None,CHD,CHKALOVSK-AVIA,Russia,N
6,6,7,224th Flight Unit,\N,None,TTF,CARGO UNIT,Russia,N
7,7,8,247 Jet Ltd,\N,None,TWF,CLOUD RUNNER,United Kingdom,N
8,8,9,3D Aviation,\N,None,SEC,SECUREX,United States,N
9,9,10,40-Mile Air,\N,Q5,MLA,MILE-AIR,United States,Y


Our cursor is what we'll use to execute queries on a database.

#### Using `Pragma`

[`Pragma`](https://www.sqlite.org/pragma.html) - note that PRAGMA is a query statement specific to SQLite

**output:**

`(column id, column name, data type, whether or not the column can be NULL, the default value for the column, and whether the column is a foreign key)`

In [5]:
cur.execute("PRAGMA table_info(airports)")
info = cur.fetchall()
print(*info, sep = "\n")  #cool new way of using python's print

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for the other two tables, `airlines` and `routes`

In [6]:
# Grab data on the airlines table
cur.execute("PRAGMA table_info(airlines)")
info = cur.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [7]:
# Grab data on the routes table
cur.execute("PRAGMA table_info(routes)")
info = cur.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Basic SQL commands 

### The Structure of a SQL Query

![sql query structure](images/sql_statement.jpg)

[Image Source](https://searchsqlserver.techtarget.com/feature/How-to-use-the-SELECT-statement-in-SQL)

### Constructing SQL queries

**`SELECT`**:  The columns you want

- options: 
    - `DISTINCT`
    - using `AS` to rename columns, called *aliasing*
    - single number aggregates (like count)

**`FROM`:** the source tables

- options: 
    - also can alias with `AS`
    - where we can join other tables, with `[LEFT|INNER|RIGHT|FULL] JOIN ___ [ON|USING]`

**`WHERE`**: your filters

- options: 
    - comparators like `=` & `>=`
    - `BETWEEN`, `IN`, `LIKE` (with wildcards `%`)
    - booleans like `AND`, `OR`, `NOT`

**`ORDER BY`**: sorting

- options: 
    - `ASC` (default) and `DESC`

**`LIMIT`**:  # of rows to return (pair with `OFFSET`)

There are more! So many more!

**NOTE:** SQL doesn't care about spacing, and doesn't care about capslock for statement options. But, it's convention - plus it makes your queries easier to read, for yourself and others.

#### Use `sqlite_master` to find all the tables in the schema

Remember, we've defined a `cursor` that can execute queries - time to use it!

In [8]:
cur .execute(""" SELECT *
                 FROM sqlite_master
                 WHERE type = 'table';
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df

3


,type,name,tbl_name,rootpage,sql
0,table,airports,airports,2,"CREATE TABLE airports (\n[index] INTEGER,\n [..."
1,table,airlines,airlines,945,"CREATE TABLE airlines (\n[index] INTEGER,\n [..."
2,table,routes,routes,1393,"CREATE TABLE routes (\n[index] INTEGER,\n [ai..."


**Task**: 

- Select only **active** airlines in the **United Kingdom** from the `airlines` table


In [9]:
cur.execute(""" SELECT *
                FROM airlines
                WHERE country = 'United Kingdom' AND active = 'Y';
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

40


,index,id,name,alias,iata,icao,callsign,country,active
0,111,112,Astraeus,\N,5W,AEU,FLYSTAR,United Kingdom,Y
1,491,492,Air Southwest,\N,None,WOW,SWALLOW,United Kingdom,Y
2,507,508,Aurigny Air Services,\N,GR,AUR,AYLINE,United Kingdom,Y
3,564,565,Air Wales,\N,6G,AWW,RED DRAGON,United Kingdom,Y
4,664,665,AD Aviation,\N,None,VUE,FLIGHTVUE,United Kingdom,Y


In [ ]:
# can also check the description for details once a query has run
 

**Task**: 
- Select the **unique** list of **countries** with airports

In [10]:
cur.execute(""" SELECT DISTINCT country
                FROM airports;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

240


,country
0,Papua New Guinea
1,Greenland
2,Iceland
3,Canada
4,Algeria


In [11]:
# what if I just wanted a count of how many?
cur.execute(""" SELECT COUNT(DISTINCT country)
                FROM airports;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df

1


,COUNT(DISTINCT country)
0,240


### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="images/venn.png" width=550>

**Task** 

- Write a query that will join the **latitude** and **longitude** data from the `airports` table to the information on the `routes` table

In [12]:
# what does the routes table look like?
cur.execute(""" SELECT *
                FROM routes;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

67663


,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


In [13]:
# what does the airports table look like?
cur.execute(""" SELECT *
                FROM airports;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

8107


,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [18]:
# what join(s) do we need?
cur.execute(""" SELECT r.*,
                        source.latitude AS source_lat,
                        source.longitude AS source_long,
                        dest.latitude AS dest_lat,
                        dest.longitude AS dest_long
                FROM routes r
                JOIN airports AS source ON r.source_id = source.id
                JOIN airports AS dest ON r.dest_id = dest.id;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

67202


,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,source_lat,source_long,dest_lat,dest_long
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,43.449928,39.956589,55.606186,49.278728
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,46.283333,48.006278,55.606186,49.278728
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,46.283333,48.006278,44.225072,43.081889
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,55.305836,61.503333,55.606186,49.278728
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,55.305836,61.503333,55.012622,82.650656


### Grouping statements

Combine `SELECT` and `GROUP BY` when you want aggregates by values

`SELECT` `min(x)` ... `max()`, `sum()`, etc.

`GROUP BY x`

**Task**

- Which countries have the most active airlines?

In [14]:
cur.execute(""" SELECT *, COUNT(*) AS active_airlines    
                FROM airlines
                WHERE active = 'Y'
                GROUP BY country
                ORDER BY active_airlines DESC;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

195


,index,id,name,alias,iata,icao,callsign,country,active,active_airlines
0,9,10,40-Mile Air,\N,Q5,MLA,MILE-AIR,United States,Y,141
1,129,130,Aeroflot Russian Airlines,\N,SU,AFL,AEROFLOT,Russia,Y,72
2,111,112,Astraeus,\N,5W,AEU,FLYSTAR,United Kingdom,Y,40
3,213,214,Air Berlin,\N,AB,BER,AIR BERLIN,Germany,Y,37
4,329,330,Air Canada,\N,AC,ACA,AIR CANADA,Canada,Y,34


**Task**

- What about inactive airlines?

In [15]:
cur.execute(""" SELECT *, COUNT(*) AS inactive_airlines    
                FROM airlines
                WHERE active = 'N'
                GROUP BY country
                ORDER BY inactive_airlines DESC;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

250


,index,id,name,alias,iata,icao,callsign,country,active,inactive_airlines
0,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N,939
1,17,18,Aero Servicios Ejecutivos Internacionales,\N,None,SII,ASEISA,Mexico,N,427
2,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N,367
3,11,12,611897 Alberta Limited,\N,None,THD,DONUT,Canada,N,284
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N,158


**Task**

- How many airports are there in each timezone?

In [16]:
cur.execute(""" SELECT timezone, COUNT(name) AS num_airports
                FROM airports
                GROUP BY timezone
                ;
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

294


,timezone,num_airports
0,Africa/Abidjan,7
1,Africa/Accra,6
2,Africa/Addis_Ababa,31
3,Africa/Algiers,44
4,Africa/Asmera,4


In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Moving from SQLite3 to pandas

In [ ]:
# need to import pandas!


In [1]:
pd_con = None
df = None

In [ ]:
df.head()

**Task**: 
Convert one of the earlier queries in the lesson to a pandas dataframe

Another way to move results into a pandas dataframe:

In [ ]:
# don't forget to close connections again

## Additional Resources

Reading Resources:

- [MariaDB's list of relational database terms, which also helps explain table relationships](https://mariadb.com/kb/en/relational-databases-basic-terms/)
- [History of SQL Article](https://www.businessnewsdaily.com/5804-what-is-sql.html)
- [The original SQL paper from the 1970s](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

Free SQL Courses: 

- [Kaggle's Courses](https://www.kaggle.com/learn/overview) on Intro to SQL and Advanced SQL - will include connecting to a Google Biq Query database
- [Khan Academy's SQL Course](https://www.khanacademy.org/computing/computer-programming/sql), which includes using more complicated query commands like CASE
- [Coursera Course on Modern Big Data Analysis with SQL](https://www.coursera.org/specializations/cloudera-big-data-analysis-sql) which was just recommended to me via the data science subreddit - covers SQL queries with specific considerations for very very large datasets stored in clusters in the cloud (specifically covers Hive and Impala, I'll likely be taking this course for fun over the next few weeks if anyone wants to join me!)